In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

In [4]:
print("📥 Loading ultra training data...")
df = pd.read_csv("D:/FinalProject/ultra_training_data.csv")
df = df[df['text'].notna() & df['emotion'].notna()]
df['emotion'] = df['emotion'].astype("category")
df['label'] = df['emotion'].cat.codes

📥 Loading ultra training data...


In [6]:
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.2, random_state=42)


In [8]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

In [10]:
def encode_data(texts, labels):
    encodings = tokenizer(texts.tolist(), padding=True, truncation=True, max_length=384)
    encodings["labels"] = labels.tolist()
    return Dataset.from_dict(encodings)


In [12]:
train_dataset = encode_data(X_train, y_train)
test_dataset = encode_data(X_test, y_test)

In [14]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    fp16=True
)

C:\Users\harip\anaconda3\Lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(pred.label_ids, preds)
    f1 = f1_score(pred.label_ids, preds, average='weighted')
    return {"accuracy": acc, "f1": f1}


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
print("🚀 Training model...")
trainer.train()


🚀 Training model...


Step,Training Loss,Validation Loss,Accuracy,F1
1000,0.000200,0.000099,1.000000,1.000000
2000,0.000100,0.000033,1.000000,1.000000
3000,0.000000,0.000016,1.000000,1.000000
4000,0.000000,0.000009,1.000000,1.000000
5000,0.000000,0.000005,1.000000,1.000000
6000,0.000000,0.000003,1.000000,1.000000
7000,0.000000,0.000002,1.000000,1.000000
8000,0.000000,0.000001,1.000000,1.000000
9000,0.000000,0.000001,1.000000,1.000000
10000,0.000000,0.000001,1.000000,1.000000


In [ ]:
model.save_pretrained("absa_model_v3")
tokenizer.save_pretrained("absa_model_v3")
print("✅ Model saved as absa_model_v3")

